#Notebook for Python 2

In [2]:
!pip install statistics

  Created wheel for statistics: filename=statistics-1.0.3.5-cp27-none-any.whl size=7452 sha256=3c50a2098000cbcf735cfdb3d08a09b096267b8e720774803787ffb330c735f5
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
Successfully built statistics


In [6]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.metrics.scores import accuracy, precision, recall, f_measure
import nltk.metrics
from nltk import accuracy, precision, recall, f_measure
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import sklearn.metrics
from sklearn import  metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import collections

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('cess_esp')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.


True

In [14]:
#adaptacion SPANISH
from nltk.corpus import cess_esp
from nltk.metrics.scores import (precision, recall)
nltk.tag.mapping._load_universal_map("es-cast3lb") 
mapdict = nltk.tag.mapping._MAPPINGS["es-cast3lb"]["universal"] 
alltags = set(t for w, t in cess_esp.tagged_words())
for tag in alltags:
    if len(tag) <= 2:   # These are complete
        continue
    mapdict[tag] = mapdict[tag[:2]]

cess_esp._tagset = "es-cast3lb"
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt
cess_sents = cess_esp.tagged_sents(tagset='universal')
uni_tag = ut(cess_sents, backoff=nltk.DefaultTagger('X'))

In [15]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

We need to upload the tranining data (hate_sexual.txt and no_hate_sexual.txt)

In [8]:
from google.colab import files
files.upload()

Saving hate_sexual.txt to hate_sexual.txt
Saving no_hate_sexual.txt to no_hate_sexual.txt


{u'hate_sexual.txt': 'Que asco esta hp:(.\r\n@c_tangana Hay que vender como sea \xf0\x9f\x91\x85,todo vale homobofo, machirulo , contrahecho\xf0\x9f\x90\xb5, y ahora tb.  CDR/PDRRO \xf0\x9f\xa4\x91Miraverrrrrrr \xf0\x9f\xa4\x93que el  Tontorra esta buscando figaflors para el fin de fiesta.., \xf0\x9f\x92\xa3\xf0\x9f\x92\xa3\r\nPuto maricon.\r\n@mashinne316 @cristian_arayal @ahausv Exacto...!! porque si yo ando haciendo cagadas, despu\xc3\xa9s aguanto como hombrecito y no ando lloriqueando como los maricones!!\xf0\x9f\xa4\xb7\xf0\x9f\x8f\xbb\xe2\x80\x8d\xe2\x99\x82\xef\xb8\x8f\xf0\x9f\xa4\xb7\xf0\x9f\x8f\xbb\xe2\x80\x8d\xe2\x99\x82\xef\xb8\x8f\r\n@kokomofans @Guillem_Andres Chupapollas\r\n@Sanguchettti @rodrigo_maron Es as\xc3\xad... uno sopla nuca el otro muerde la almohada. El Scioli de las almohadas\r\n@TVyNovelasMex Maldita lesbiana!! El mal.ejemplo de mexico\r\nRaja de ac\xc3\xa1 machirulo asqueroso\r\nPi\xc3\xb1era putoPi\xc3\xb1era culiaoPi\xc3\xb1era sacoweaPi\xc3\xb1era chupapi

In [16]:
!ls
!wc no_hate_sexual.txt
!wc hate_sexual.txt

hate_sexual.txt  no_hate_sexual.txt  sample_data
  8081 136596 862351 no_hate_sexual.txt
  2772  49296 316263 hate_sexual.txt


In [17]:
short_pos = open("no_hate_sexual.txt","r").read()
short_neg = open("hate_sexual.txt","r").read()

short_pos = short_pos.decode('utf-8', 'ignore')
short_neg = short_neg.decode('utf-8', 'ignore')

In [18]:
short_neg

u'Que asco esta hp:(.\r\n@c_tangana Hay que vender como sea \U0001f445,todo vale homobofo, machirulo , contrahecho\U0001f435, y ahora tb.  CDR/PDRRO \U0001f911Miraverrrrrrr \U0001f913que el  Tontorra esta buscando figaflors para el fin de fiesta.., \U0001f4a3\U0001f4a3\r\nPuto maricon.\r\n@mashinne316 @cristian_arayal @ahausv Exacto...!! porque si yo ando haciendo cagadas, despu\xe9s aguanto como hombrecito y no ando lloriqueando como los maricones!!\U0001f937\U0001f3fb\u200d\u2642\ufe0f\U0001f937\U0001f3fb\u200d\u2642\ufe0f\r\n@kokomofans @Guillem_Andres Chupapollas\r\n@Sanguchettti @rodrigo_maron Es as\xed... uno sopla nuca el otro muerde la almohada. El Scioli de las almohadas\r\n@TVyNovelasMex Maldita lesbiana!! El mal.ejemplo de mexico\r\nRaja de ac\xe1 machirulo asqueroso\r\nPi\xf1era putoPi\xf1era culiaoPi\xf1era sacoweaPi\xf1era chupapicoPi\xf1era bastardoPi\xf1era tula chicaPi\xf1era mariconPi\xf1era conchetumarePi\xf1era flopPi\xf1era simioPi\xf1era clownPi\xf1era hijo de la 

In [19]:
all_words = []
documents = []

In [20]:
allowed_word_types = ['NOUN', 'ADJ', 'VERB']

for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = nltk.word_tokenize(p)
    pos = uni_tag.tag(words)
    for w in pos:
        if w[1] in allowed_word_types:
            all_words.append(w[0].lower())

    
for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = nltk.word_tokenize(p)
    pos = uni_tag.tag(words)
    for w in pos:
        if w[1] in allowed_word_types:
            all_words.append(w[0].lower())

In [21]:
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:10000] #5000

In [22]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)
print(len(featuresets))

10855


In [23]:
training_set = featuresets[:5000]
testing_set = featuresets[5000:]

In [24]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
##Para metricas
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

#cross_valid_accuracy = nltk.classify.accuracy(classifier, cross_valid_set)
for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
print('Precision:', nltk.precision(refsets['pos'], testsets['pos']))
print('Recall:', nltk.recall(refsets['pos'], testsets['pos']))
print('f_measure:', nltk.f_measure(refsets['pos'], testsets['pos']))


# Precision-Recall AUC
def label2int(label):
    if label == 'pos':
        return 1
    else:
        return 0

y_true, y_score = [], []

for i, (feats, label_true) in enumerate(testing_set):
    label_predicted = classifier.classify(feats)
    y_true.append(label2int(label_true))
    y_score.append(label2int(label_predicted))

precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_auc = auc(recall, precision)
print ("Precision-Recall AUC: %.2f" % pr_auc)
# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: %.2f" % roc_auc)

#
classifier.show_most_informative_features(50)

('Original Naive Bayes Algo accuracy percent:', 73.9538855678907)
('Precision:', 0.7966813694601974)
('Recall:', 0.8719540229885058)
('f_measure:', 0.8326199099989025)
Precision-Recall AUC: 0.88
ROC AUC: 0.61
Most Informative Features
                 maldito = True              neg : pos    =     24.5 : 1.0
                     amo = True              pos : neg    =     17.1 : 1.0
               almohadas = True              neg : pos    =     13.5 : 1.0
                 imbécil = True              neg : pos    =     12.7 : 1.0
                valiente = True              neg : pos    =     10.8 : 1.0
                    vete = True              neg : pos    =     10.8 : 1.0
                deberían = True              neg : pos    =     10.8 : 1.0
                 muertos = True              neg : pos    =     10.0 : 1.0
                  barrio = True              neg : pos    =      8.8 : 1.0
                  perros = True              neg : pos    =      8.8 : 1.0
               

In [25]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = MNB_classifier.classify(feats)
    testsets[observed].add(i)

print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)
print('Precision:', nltk.precision(refsets['pos'], testsets['pos']))
print('Recall:', nltk.recall(refsets['pos'], testsets['pos']))
print('f_measure:', nltk.f_measure(refsets['pos'], testsets['pos']))

# Precision-Recall AUC
def label2int(label):
    if label == 'pos':
        return 1
    else:
        return 0

y_true, y_score = [], []

for i, (feats, label_true) in enumerate(testing_set):
    label_predicted = MNB_classifier.classify(feats)
    y_true.append(label2int(label_true))
    y_score.append(label2int(label_predicted))

precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_auc = auc(recall, precision)
print ("Precision-Recall AUC: %.2f" % pr_auc)
# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: %.2f" % roc_auc)

#


BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = BernoulliNB_classifier.classify(feats)
    testsets[observed].add(i)

print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)
print('Precision:', nltk.precision(refsets['pos'], testsets['pos']))
print('Recall:', nltk.recall(refsets['pos'], testsets['pos']))
print('f_measure:', nltk.f_measure(refsets['pos'], testsets['pos']))

# Precision-Recall AUC
def label2int(label):
    if label == 'pos':
        return 1
    else:
        return 0

y_true, y_score = [], []

for i, (feats, label_true) in enumerate(testing_set):
    label_predicted = BernoulliNB_classifier.classify(feats)
    y_true.append(label2int(label_true))
    y_score.append(label2int(label_predicted))

precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_auc = auc(recall, precision)
print ("Precision-Recall AUC: %.2f" % pr_auc)
# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: %.2f" % roc_auc)

#

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = LogisticRegression_classifier.classify(feats)
    testsets[observed].add(i)

print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)
print('Precision:', nltk.precision(refsets['pos'], testsets['pos']))
print('Recall:', nltk.recall(refsets['pos'], testsets['pos']))
print('f_measure:', nltk.f_measure(refsets['pos'], testsets['pos']))

# Precision-Recall AUC
def label2int(label):
    if label == 'pos':
        return 1
    else:
        return 0

y_true, y_score = [], []

for i, (feats, label_true) in enumerate(testing_set):
    label_predicted = LogisticRegression_classifier.classify(feats)
    y_true.append(label2int(label_true))
    y_score.append(label2int(label_predicted))

precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_auc = auc(recall, precision)
print ("Precision-Recall AUC: %.2f" % pr_auc)
# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: %.2f" % roc_auc)

#

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = LinearSVC_classifier.classify(feats)
    testsets[observed].add(i)

print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)
print('Precision:', nltk.precision(refsets['pos'], testsets['pos']))
print('Recall:', nltk.recall(refsets['pos'], testsets['pos']))
print('f_measure:', nltk.f_measure(refsets['pos'], testsets['pos']))

# Precision-Recall AUC
def label2int(label):
    if label == 'pos':
        return 1
    else:
        return 0

y_true, y_score = [], []

for i, (feats, label_true) in enumerate(testing_set):
    label_predicted = LinearSVC_classifier.classify(feats)
    y_true.append(label2int(label_true))
    y_score.append(label2int(label_predicted))

precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_auc = auc(recall, precision)
print ("Precision-Recall AUC: %.2f" % pr_auc)
# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: %.2f" % roc_auc)

#

SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = SGDC_classifier.classify(feats)
    testsets[observed].add(i)

print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)
print('Precision:', nltk.precision(refsets['pos'], testsets['pos']))
print('Recall:', nltk.recall(refsets['pos'], testsets['pos']))
print('f_measure:', nltk.f_measure(refsets['pos'], testsets['pos']))

# Precision-Recall AUC
def label2int(label):
    if label == 'pos':
        return 1
    else:
        return 0

y_true, y_score = [], []

for i, (feats, label_true) in enumerate(testing_set):
    label_predicted = SGDC_classifier.classify(feats)
    y_true.append(label2int(label_true))
    y_score.append(label2int(label_predicted))

precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_auc = auc(recall, precision)
print ("Precision-Recall AUC: %.2f" % pr_auc)
# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: %.2f" % roc_auc)

#


voted_classifier = VoteClassifier(
                                  classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("Voted classifier:",nltk.classify.accuracy(voted_classifier, testing_set)*100)
print('Precision:', nltk.precision(refsets['pos'], testsets['pos']))
print('Recall:', nltk.recall(refsets['pos'], testsets['pos']))
print('f_measure:', nltk.f_measure(refsets['pos'], testsets['pos']))

# Precision-Recall AUC
def label2int(label):
    if label == 'pos':
        return 1
    else:
        return 0

y_true, y_score = [], []

for i, (feats, label_true) in enumerate(testing_set):
    label_predicted = voted_classifier.classify(feats)
    y_true.append(label2int(label_true))
    y_score.append(label2int(label_predicted))

precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_auc = auc(recall, precision)
print ("Precision-Recall AUC: %.2f" % pr_auc)
# ROC AUC
fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
print("ROC AUC: %.2f" % roc_auc)



('MNB_classifier accuracy percent:', 76.43040136635354)
('Precision:', 0.7850287907869482)
('Recall:', 0.9402298850574713)
('f_measure:', 0.8556485355648537)
Precision-Recall AUC: 0.88
ROC AUC: 0.60
('BernoulliNB_classifier accuracy percent:', 74.43210930828353)
('Precision:', 0.7659891851575611)
('Recall:', 0.944367816091954)
('f_measure:', 0.8458766601461957)
Precision-Recall AUC: 0.88
ROC AUC: 0.56


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


('LogisticRegression_classifier accuracy percent:', 78.54824935952178)
('Precision:', 0.7993421052631579)
('Recall:', 0.9496551724137932)
('f_measure:', 0.8680395040974994)
Precision-Recall AUC: 0.89
ROC AUC: 0.63
('LinearSVC_classifier accuracy percent:', 75.5593509820666)
('Precision:', 0.8057825267127593)
('Recall:', 0.8841379310344828)
('f_measure:', 0.8431437027293652)
Precision-Recall AUC: 0.89
ROC AUC: 0.63


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


('SGDClassifier accuracy percent:', 75.73014517506405)
('Precision:', 0.8094231988168181)
('Recall:', 0.8806896551724138)
('f_measure:', 0.8435538918859409)
Precision-Recall AUC: 0.89
ROC AUC: 0.64
('Voted classifier:', 76.9598633646456)
('Precision:', 0.8094231988168181)
('Recall:', 0.8806896551724138)
('f_measure:', 0.8435538918859409)
Precision-Recall AUC: 0.89
ROC AUC: 0.61
